<article class="day-desc"><h2>--- Day 5: Cafeteria ---</h2><p>As the forklifts break through the wall, the Elves are delighted to discover that there was a cafeteria on the other side after all.</p>
<p>You can hear a commotion coming from the kitchen. "At this rate, we won't have any time left to put the wreaths up in the dining hall!" Resolute in your quest, you investigate.</p>
<p>"If only we hadn't switched to the new inventory management system right before Christmas!" another Elf exclaims. You ask what's going on.</p>
<p>The Elves in the kitchen explain the situation: because of their complicated new inventory management system, they can't figure out which of their ingredients are <em>fresh</em> and which are <span title="No, this puzzle does not take place on Gleba. Why do you ask?"><em>spoiled</em></span>. When you ask how it works, they give you a copy of their database (your puzzle input).</p>
<p>The database operates on <em>ingredient IDs</em>. It consists of a list of <em>fresh ingredient ID ranges</em>, a blank line, and a list of <em>available ingredient IDs</em>. For example:</p>
<pre><code>3-5
10-14
16-20
12-18

1
5
8
11
17
32
</code></pre>
<p>The fresh ID ranges are <em>inclusive</em>: the range <code>3-5</code> means that ingredient IDs <code>3</code>, <code>4</code>, and <code>5</code> are all <em>fresh</em>. The ranges can also <em>overlap</em>; an ingredient ID is fresh if it is in <em>any</em> range.</p>
<p>The Elves are trying to determine which of the <em>available ingredient IDs</em> are <em>fresh</em>. In this example, this is done as follows:</p>
<ul>
<li>Ingredient ID <code>1</code> is spoiled because it does not fall into any range.</li>
<li>Ingredient ID <code>5</code> is <em>fresh</em> because it falls into range <code>3-5</code>.</li>
<li>Ingredient ID <code>8</code> is spoiled.</li>
<li>Ingredient ID <code>11</code> is <em>fresh</em> because it falls into range <code>10-14</code>.</li>
<li>Ingredient ID <code>17</code> is <em>fresh</em> because it falls into range <code>16-20</code> as well as range <code>12-18</code>.</li>
<li>Ingredient ID <code>32</code> is spoiled.</li>
</ul>
<p>So, in this example, <em><code>3</code></em> of the available ingredient IDs are fresh.</p>
<p>Process the database file from the new inventory management system. <em>How many of the available ingredient IDs are fresh?</em></p>
</article>

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from aocd import get_data

puzzle_input = get_data(day=5, year=2025)
puzzle_input[:20]

'263168346238540-2637'

In [3]:
sample_input = """3-5
10-14
16-20
12-18

1
5
8
11
17
32"""

sample_input

'3-5\n10-14\n16-20\n12-18\n\n1\n5\n8\n11\n17\n32'

In [4]:
from dataclasses import dataclass
from typing import Self


@dataclass(frozen=True, slots=True)
class Database:
    ranges: list[tuple[int, int]]
    values: list[int]

    @classmethod
    def from_string(cls, input: str) -> Self:
        ranges_str, values_str = input.strip().split("\n\n")
        ranges: list[tuple[int, int]] = []
        for line in ranges_str.splitlines():
            parts = line.split("-")
            ranges.append((int(parts[0]), int(parts[1])))
        values = [int(line) for line in values_str.splitlines()]
        return cls(ranges, values)


Database.from_string(sample_input)

Database(ranges=[(3, 5), (10, 14), (16, 20), (12, 18)], values=[1, 5, 8, 11, 17, 32])

In [5]:
def is_fresh(value: int, ranges: list[tuple[int, int]]) -> bool:
    for r in ranges:
        if r[0] <= value <= r[1]:
            return True
    return False


is_fresh(8, [(3, 5), (10, 14), (16, 20), (12, 18)])

False

In [ ]:
def count_fresh(db: Database) -> int:
    return sum(1 for v in db.values if is_fresh(v, db.ranges))


assert count_fresh(Database.from_string(sample_input)) == 3
count_fresh(Database.from_string(sample_input))

3

In [7]:
part1 = count_fresh(Database.from_string(puzzle_input))
part1

643

<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>The Elves start bringing their spoiled inventory to the trash chute at the back of the kitchen.</p>
<p>So that they can stop bugging you when they get new inventory, the Elves would like to know <em>all</em> of the IDs that the <em>fresh ingredient ID ranges</em> consider to be <em>fresh</em>. An ingredient ID is still considered fresh if it is in any range.</p>
<p>Now, the second section of the database (the available ingredient IDs) is irrelevant. Here are the fresh ingredient ID ranges from the above example:</p>
<pre><code>3-5
10-14
16-20
12-18
</code></pre>
<p>The ingredient IDs that these ranges consider to be fresh are <code>3</code>, <code>4</code>, <code>5</code>, <code>10</code>, <code>11</code>, <code>12</code>, <code>13</code>, <code>14</code>, <code>15</code>, <code>16</code>, <code>17</code>, <code>18</code>, <code>19</code>, and <code>20</code>. So, in this example, the fresh ingredient ID ranges consider a total of <em><code>14</code></em> ingredient IDs to be fresh.</p>
<p>Process the database file again. <em>How many ingredient IDs are considered to be fresh according to the fresh ingredient ID ranges?</em></p>
</article>

In [8]:
def merge_ranges(ranges: list[tuple[int, int]]) -> list[tuple[int, int]]:
    """Merge overlapping/adjacent ranges into non-overlapping intervals."""
    if not ranges:
        return []

    sorted_ranges = sorted(ranges)
    merged: list[tuple[int, int]] = [sorted_ranges[0]]

    for start, end in sorted_ranges[1:]:
        last_start, last_end = merged[-1]
        if start <= last_end + 1:  # overlapping or adjacent
            merged[-1] = (last_start, max(last_end, end))
        else:
            merged.append((start, end))

    return merged


def count_fresh_ids(db: Database) -> int:
    merged = merge_ranges(db.ranges)
    return sum(end - start + 1 for start, end in merged)


assert count_fresh_ids(Database.from_string(sample_input)) == 14
count_fresh_ids(Database.from_string(sample_input))

14

In [9]:
part2 = count_fresh_ids(Database.from_string(puzzle_input))
part2

342018167474526